# 02 — Fixtures & Parametrize

This notebook covers:
- `@pytest.fixture` with scopes
- Built-in fixtures: `tmp_path`, `capsys`, `monkeypatch`
- `conftest.py` for shared fixtures
- Advanced parametrize patterns
- Fixture factories

## Setup

In [ ]:
import subprocess, sys, textwrap, tempfile, pathlib

def run_pytest(test_code: str, extra_args: list[str] | None = None, extra_files: dict[str, str] | None = None):
    with tempfile.TemporaryDirectory() as td:
        td_path = pathlib.Path(td)
        p = td_path / "test_tmp.py"
        p.write_text(textwrap.dedent(test_code))
        if extra_files:
            for name, content in extra_files.items():
                (td_path / name).write_text(textwrap.dedent(content))
        cmd = [sys.executable, "-m", "pytest", str(td), "-v", "--tb=short", "--no-header"]
        if extra_args:
            cmd.extend(extra_args)
        result = subprocess.run(cmd, capture_output=True, text=True, timeout=30)
        print(result.stdout + result.stderr)
        return result.returncode

print("Helper ready.")

## 1. Basic Fixtures

In [ ]:
rc = run_pytest('''
import pytest

@pytest.fixture
def sample_list():
    """Provides a fresh list for each test."""
    return [1, 2, 3, 4, 5]

def test_length(sample_list):
    assert len(sample_list) == 5

def test_sum(sample_list):
    assert sum(sample_list) == 15

def test_mutate(sample_list):
    sample_list.append(6)
    assert len(sample_list) == 6  # 6 here...

def test_still_fresh(sample_list):
    assert len(sample_list) == 5  # ...but 5 here — fixture is recreated
''')
assert rc == 0
print("Each test gets a FRESH copy of the fixture (function scope by default).")

## 2. Fixture with Teardown (`yield`)

In [ ]:
rc = run_pytest('''
import pytest

log = []

@pytest.fixture
def resource():
    log.append("setup")
    yield "the_resource"
    log.append("teardown")

def test_one(resource):
    assert resource == "the_resource"
    log.append("test_one_ran")

def test_two(resource):
    assert resource == "the_resource"
    log.append("test_two_ran")

def test_log():
    # After test_one: setup, test_one_ran, teardown
    # After test_two: setup, test_two_ran, teardown
    assert log == ["setup", "test_one_ran", "teardown", "setup", "test_two_ran", "teardown"]
''')
assert rc == 0
print("yield separates setup from teardown — cleanup is guaranteed.")

## 3. Fixture Scopes

In [ ]:
rc = run_pytest('''
import pytest

call_count = {"func": 0, "mod": 0}

@pytest.fixture
def func_fixture():
    call_count["func"] += 1
    return call_count["func"]

@pytest.fixture(scope="module")
def mod_fixture():
    call_count["mod"] += 1
    return call_count["mod"]

def test_a(func_fixture, mod_fixture):
    assert func_fixture == 1  # fresh per test
    assert mod_fixture == 1   # shared across module

def test_b(func_fixture, mod_fixture):
    assert func_fixture == 2  # incremented again
    assert mod_fixture == 1   # still 1 — module scope
''')
assert rc == 0
print("function scope: per test. module scope: once per file. session scope: once per run.")

## 4. Built-in Fixture: `tmp_path`

In [ ]:
rc = run_pytest('''
def test_write_file(tmp_path):
    # tmp_path is a pathlib.Path to a unique temp directory
    f = tmp_path / "data.txt"
    f.write_text("hello world")
    assert f.read_text() == "hello world"
    assert f.exists()

def test_csv(tmp_path):
    csv_path = tmp_path / "data.csv"
    csv_path.write_text("name,age" + chr(10) + "Alice,30" + chr(10) + "Bob,25")
    lines = csv_path.read_text().strip().splitlines()
    assert len(lines) == 3  # header + 2 rows
''')
assert rc == 0
print("tmp_path: auto-cleaned temp dir, unique per test.")

## 5. Built-in Fixture: `monkeypatch`

In [ ]:
rc = run_pytest('''
import os

def get_env_setting():
    return os.environ.get("MY_SETTING", "default")

def test_default():
    assert get_env_setting() == "default"

def test_override(monkeypatch):
    monkeypatch.setenv("MY_SETTING", "custom_value")
    assert get_env_setting() == "custom_value"

def test_still_default():
    # monkeypatch auto-reverts after the test
    assert get_env_setting() == "default"
''')
assert rc == 0
print("monkeypatch sets env vars, attributes, dict items — auto-reverted after test.")

## 6. `conftest.py` — Shared Fixtures

In [ ]:
conftest_code = '''
import pytest

@pytest.fixture
def sample_texts():
    return ["hello world", "foo bar", "test input"]

@pytest.fixture
def empty_text():
    return ""
'''

test_code = '''
def test_texts_count(sample_texts):
    assert len(sample_texts) == 3

def test_empty(empty_text):
    assert empty_text == ""
'''

rc = run_pytest(test_code, extra_files={"conftest.py": conftest_code})
assert rc == 0
print("Fixtures from conftest.py are auto-discovered — no imports needed.")

## 7. Advanced Parametrize

In [ ]:
rc = run_pytest('''
import pytest

# Using pytest.param for IDs and markers
@pytest.mark.parametrize("text, expected", [
    pytest.param("hello world", 2, id="two_words"),
    pytest.param("", 0, id="empty"),
    pytest.param("one", 1, id="single_word"),
    pytest.param("  spaces  between  ", 2, id="extra_spaces"),
])
def test_word_count(text, expected):
    words = text.split() if text.strip() else []
    assert len(words) == expected

# Stacking parametrize for combinations
@pytest.mark.parametrize("a", [1, 2])
@pytest.mark.parametrize("b", [10, 20])
def test_multiply(a, b):
    assert a * b > 0  # 4 combinations: (1,10), (1,20), (2,10), (2,20)
''')
assert rc == 0
print("pytest.param(id=...) makes output readable. Stacked parametrize = combinatorial.")

## 8. Fixture Factories

In [ ]:
rc = run_pytest('''
import pytest

@pytest.fixture
def make_user():
    """Factory fixture: returns a function that creates users."""
    created = []
    def _make(name="Alice", age=30):
        user = {"name": name, "age": age}
        created.append(user)
        return user
    yield _make
    # teardown: clean up all created users
    created.clear()

def test_default_user(make_user):
    user = make_user()
    assert user == {"name": "Alice", "age": 30}

def test_custom_user(make_user):
    user = make_user("Bob", 25)
    assert user["name"] == "Bob"

def test_multiple(make_user):
    u1 = make_user("A", 1)
    u2 = make_user("B", 2)
    assert u1 != u2
''')
assert rc == 0
print("Factory fixtures return callables — flexible and reusable.")

## Key Takeaways

1. **Fixtures** = dependency injection for tests. Request only what you need.
2. **Scopes**: `function` (default, safe) → `module` → `session` (fast, shared)
3. **`yield`** for setup+teardown in one place
4. **Built-ins**: `tmp_path`, `capsys`, `monkeypatch` — use them!
5. **`conftest.py`** shares fixtures without imports
6. **Factories** for when you need multiple instances per test